# 🧪 Lab Supply Calculator

Welcome! Use this tool to determine the quantity of supplies you will need for a given sample size, compare that to the current inventory list, and generate a report of supply needs and reorder status

🔒 **Note**: You'll need a GitHub access token to access the files. Please see Aubrie or Vicki for this.

---

## 🧭 Instructions

1. Press "Run all" to get started
2. Enter your **GitHub token**
3. Enter the **sample size**
4. Choose the **protocol**
5. Download the CSV report when prompted


In [ ]:
#@title Install packages and imports
##Install packages and imports
# Installing packages
import subprocess
import sys

def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

try:
    import yaml
except ImportError:
    install_package("pyyaml")
import yaml

try:
    import gspread
except ImportError:
    install_package("gspread")

# Ensure these are installed for Google Colab authentication with gspread
try:
    import google.auth
except ImportError:
    install_package("google-auth")
try:
    import google.auth.transport.requests
except ImportError:
    install_package("google-auth-httplib2") # or google-auth-oauthlib, httplib2 is more common for initial setup


# Import for Google Colab authentication
try:
    from google.colab import auth
except ImportError:
    print("Warning: google.colab.auth not found. Running outside Colab might require manual authentication setup for gspread.")


# Import all required libraries
import requests
import csv
import pandas as pd
import os
from io import StringIO
from google.colab import files
from IPython.display import display, clear_output
import ipywidgets as widgets
import time

print("✅ All packages installed and imported successfully!")

# --- Google Sheet Authentication Sanity Check (Moved Here) ---
global gc # Make gc global so it can be used in load_inventory
try:
    print("\n🔑 Authenticating Google Colab for Sheets access (this may open a pop-up)...")
    auth.authenticate_user()
    creds, _ = google.auth.default()
    gc = gspread.authorize(creds)
    print("✅ Google Sheets authentication successful!")
except Exception as e:
    print(f"❌ Initial Google Sheets authentication failed: {e}")
    print("Please ensure you complete the authentication pop-up and have proper sheet permissions.")
    gc = None # Set gc to None if authentication fails
# ----------------------------------------------------------

# Define GitHub repo configuration
GITHUB_USER = "aono87"
REPO_NAME = "inventory_test"
BRANCH = "main"
# INVENTORY_PATH is no longer directly used for Google Sheet
PROTOCOL_DIR = "test_protocol_check/protocols"
GOOGLE_SHEET_URL = "https://docs.google.com/spreadsheets/d/1uJeollRVlBDNcQU-FnPCYjLaCHbbOND2OEqcm3eKd2M/edit?usp=sharing"

print("✅ Configuration set")

#Helper Functions
def github_raw_url(path):
    return f"https://raw.githubusercontent.com/{GITHUB_USER}/{REPO_NAME}/{BRANCH}/{path}"

def fetch_file(path, token):
    url = github_raw_url(path)
    headers = {"Authorization": f"token {token}"}
    try:
        r = requests.get(url, headers=headers)
        r.raise_for_status()
        return r.text
    except requests.exceptions.RequestException as e:
        print(f"❌ Error fetching {path}: {e}")
        raise

def get_protocol_list(token, user, repo, path, branch="main"):
    url = f"https://api.github.com/repos/{user}/{repo}/contents/{path}?ref={branch}"
    headers = {"Authorization": f"token {token}"}
    try:
        r = requests.get(url, headers=headers)
        r.raise_for_status()
        files = r.json()
        return [f['path'] for f in files if f['name'].endswith('.yaml')]
    except requests.exceptions.RequestException as e:
        print(f"❌ Error fetching protocol list: {e}")
        return []

def load_inventory():
    """
    Loads inventory data from the specified Google Sheet.
    Assumes the first worksheet contains the inventory data.
    """
    if gc is None:
        print("❌ Google Sheets client not initialized. Cannot load inventory.")
        return {}

    try:
        spreadsheet = gc.open_by_url(GOOGLE_SHEET_URL)
        worksheet = spreadsheet.get_worksheet(0)  # Get the first worksheet

        # Get all records as a list of dictionaries
        records = worksheet.get_all_records()

        inv = {}
        for row in records:
            item = row.get('Item', '').strip()
            if not item: # Skip rows that don't have an item name
                continue
            try:
                inv[item] = {
                    'unit': row.get('Unit', '').strip(),
                    'stock': float(row.get('Stock Quantity', 0)),
                    'threshold': float(row.get('Reorder Threshold', 0))
                }
            except ValueError as ve:
                print(f"⚠️ Skipping row due to data conversion error for item '{item}': {ve}")
                continue
        print("✅ Inventory loaded successfully from Google Sheet.")
        return inv
    except Exception as e:
        print(f"❌ Error loading inventory from Google Sheet: {e}")
        print("Please ensure the Google Sheet is shared correctly (e.g., 'Anyone with the link can view').")
        return {}

def load_protocol(text):
    try:
        return yaml.safe_load(text)
    except Exception as e:
        print(f"❌ Error loading protocol: {e}")
        return {}

def calculate_needs(protocol_data, sample_count):
    if 'supplies_per_sample' not in protocol_data:
        print("❌ Protocol missing 'supplies_per_sample' section")
        return {}
    return {item: qty * sample_count for item, qty in protocol_data['supplies_per_sample'].items()}

print("✅ Helper functions defined")

In [ ]:
#@title Input github token and sample size
print("📝 Please enter your details below:")

# Create input widgets
token_input = widgets.Text(
    value='',
    placeholder='Enter your GitHub token here (ghp_...)',
    description='GitHub token:',
    layout=widgets.Layout(width='80%'),
    style={'description_width': 'initial'}
)

sample_input = widgets.IntText(
    value=50,
    description='Sample size:',
    disabled=False,
    style={'description_width': 'initial'}
)

# Create a button to proceed
proceed_button = widgets.Button(
    description='Continue with these settings',
    disabled=False,
    button_style='success',
    tooltip='Click to proceed with the entered token and sample size',
    layout=widgets.Layout(width='300px', margin='10px 0px')
)

# Status output
status_output = widgets.Output()

# Container for the next step widgets
next_step_container = widgets.Output()

def process_inventory_check():
    """Main processing function that handles everything"""

    print("✅ Inputs confirmed!")
    print(f"   Token: {token_input.value[:4]}...")
    print(f"   Sample size: {sample_input.value}")

    try:
        # Step 1: Fetch available protocols
        print("\n🔍 Step 1: Fetching available protocols...")
        protocols = get_protocol_list(
            token_input.value,
            GITHUB_USER,
            REPO_NAME,
            PROTOCOL_DIR,
            BRANCH
        )

        if not protocols:
            print("❌ No protocols found, using defaults")
            protocols = [
                "test_protocol_check/protocols/dna_extraction_mn.yaml",
                "test_protocol_check/protocols/pcr_setup.yaml"
            ]
        else:
            print(f"✅ Found {len(protocols)} protocols")

        # Step 2: Create protocol dropdown
        print("\n🔧 Step 2: Creating protocol selector...")
        protocol_dropdown = widgets.Dropdown(
            options=protocols,
            description='Protocol:',
            layout=widgets.Layout(width='95%'),
            style={'description_width': 'initial'}
        )

        print("✅ Protocol dropdown created!")
        print("👆 Please select your protocol above")

        # Step 3: Create processing button
        process_button = widgets.Button(
            description='Generate Report',
            disabled=False,
            button_style='primary',
            tooltip='Click to generate the supply report',
            layout=widgets.Layout(width='200px', margin='10px 0px')
        )

        # Output area for results
        output_area = widgets.Output()

        def generate_report(b):
            """Generate the supply report"""
            with output_area:
                output_area.clear_output()

                try:
                    print("🔄 Generating report...")

                    # Fetch data files
                    print("📥 Fetching inventory data from Google Sheet...")
                    # inventory_text = fetch_file(INVENTORY_PATH, token_input.value) # No longer needed
                    inventory = load_inventory() # Call the modified load_inventory without arguments

                    print("📥 Fetching protocol data from GitHub...")
                    protocol_text = fetch_file(protocol_dropdown.value, token_input.value)


                    # Parse data
                    print("🔍 Parsing data...")
                    # inventory = load_inventory(inventory_text) # Replaced by direct call above
                    protocol = load_protocol(protocol_text)

                    if not inventory:
                        raise Exception("Failed to load inventory data from Google Sheet")
                    if not protocol:
                        raise Exception("Failed to load protocol data")

                    # Calculate needs
                    needs = calculate_needs(protocol, sample_input.value)

                    if not needs:
                        raise Exception("Failed to calculate supply needs")

                    print(f"✅ Calculated needs for {len(needs)} items")

                    # Create report
                    report_rows = []
                    per_sample = protocol.get("supplies_per_sample", {})

                    for item, total_required in needs.items():
                        per_unit = per_sample.get(item, "N/A")
                        inv = inventory.get(item)

                        if not inv:
                            report_rows.append({
                                "Item": item,
                                "Per Sample": per_unit,
                                "Need": total_required,
                                "Stock": "N/A",
                                "Status": "MISSING",
                                "Reorder": "⚠️"
                            })
                        else:
                            stock = inv['stock']
                            threshold = inv['threshold']
                            status = "OK" if stock >= total_required else "LOW"
                            reorder_flag = "YES" if (stock - total_required) < threshold else "NO"
                            report_rows.append({
                                "Item": item,
                                "Per Sample": per_unit,
                                "Need": round(total_required, 2),
                                "Stock": round(stock, 2),
                                "Status": status,
                                "Reorder": reorder_flag
                            })

                    # Convert to DataFrame
                    df = pd.DataFrame(report_rows)

                    # Display report
                    protocol_name = os.path.basename(protocol_dropdown.value)
                    print(f"\n📋 Supply Report for {sample_input.value} samples using {protocol_name}")
                    print("=" * 80)
                    display(df)

                    # Generate and download CSV
                    print("\n💾 Generating CSV download...")

                    # Create metadata
                    metadata_info = [
                        ["Protocol:", protocol_name],
                        ["Sample Size:", sample_input.value],
                        ["Generated:", pd.Timestamp.now().strftime("%Y-%m-%d %H:%M:%S")],
                        [""], # Blank row
                    ]

                    # Save to CSV
                    output_path = "/content/supply_report.csv"

                    with open(output_path, 'w', newline='', encoding='utf-8') as f:
                        writer = csv.writer(f)
                        for row in metadata_info:
                            writer.writerow(row)

                    # Append DataFrame
                    df.to_csv(output_path, mode='a', index=False)

                    print("✅ Report generated successfully!")
                    print("📁 Starting download...")

                    # Download file
                    files.download(output_path)
                    print("🎉 Download complete!")

                    # Update button
                    process_button.description = "Report Generated ✓"
                    process_button.button_style = "success"
                    process_button.disabled = True

                except Exception as e:
                    print(f"❌ Error: {e}")
                    print("Please check your inputs and try again.")

        # Connect button to function
        process_button.on_click(generate_report)

        # Display protocol selector, button and output area
        display(protocol_dropdown, process_button, output_area)

        print("\n✅ Everything is ready!")
        print("👆 Select your protocol and click 'Generate Report' to continue")

        return True

    except Exception as e:
        print(f"❌ Setup error: {e}")
        return False

# Function to handle button click
def on_button_click(b):
    with status_output:
        status_output.clear_output()
        if not token_input.value or len(token_input.value) < 10:
            print("❌ Please enter a valid GitHub token")
            return
        if sample_input.value <= 0:
            print("❌ Please enter a valid sample size")
            return
        print(f"✅ Token received: {token_input.value[:4]}...")
        print(f"✅ Sample size: {sample_input.value}")
        print("✅ Ready to proceed to next step!")
        proceed_button.disabled = True
        proceed_button.description = "Settings confirmed ✓"

        # Clear the next step container and run the processing
        with next_step_container:
            next_step_container.clear_output()
            print("🚀 Starting Lab Supply Calculator...")
            print("=" * 50)
            success = process_inventory_check()
            if not success:
                print("\n⏸️  Process failed - please check your inputs")

proceed_button.on_click(on_button_click)

# Display all widgets
display(token_input, sample_input, proceed_button, status_output, next_step_container)

print("👆 Enter your GitHub token and sample size above, then click 'Continue' to proceed")
print("🔄 The protocol selection will appear automatically after you confirm your settings!")